In [ ]:
import cv2
import numpy as np
import glob
from matplotlib import pyplot as plt
from google.colab import drive
import os
import json

In [ ]:
# Only needed when using Google Colab
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def get_pixel(img, center, x, y):
    new_value = 0

    try:
        # If local neighbourhood pixel
        # value is greater than or equal
        # to center pixel values then
        # set it to 1
        if img[x][y] >= center:
            new_value = 1

    except:
        # Exception is required when
        # neighbourhood value of a center
        # pixel value is null i.e. values
        # present at boundaries.
        pass

    return new_value

In [ ]:
# Function for calculating LBP
def lbp_calculated_pixel(img, x, y):
    center = img[x][y]

    val_ar = []

    # top_left
    val_ar.append(get_pixel(img, center, x - 1, y - 1))

    # top
    val_ar.append(get_pixel(img, center, x - 1, y))

    # top_right
    val_ar.append(get_pixel(img, center, x - 1, y + 1))

    # right
    val_ar.append(get_pixel(img, center, x, y + 1))

    # bottom_right
    val_ar.append(get_pixel(img, center, x + 1, y + 1))

    # bottom
    val_ar.append(get_pixel(img, center, x + 1, y))

    # bottom_left
    val_ar.append(get_pixel(img, center, x + 1, y - 1))

    # left
    val_ar.append(get_pixel(img, center, x, y - 1))

    # Now, we need to convert binary
    # values to decimal
    power_val = [1, 2, 4, 8, 16, 32, 64, 128]

    val = 0

    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]

    return val

In [ ]:
def generate_histogram(img):
  result = []
  img_bgr = cv2.imread(img, 1)
  # breakpoint()
  height, width, _ = img_bgr.shape
  # We need to convert RGB image
  # into gray one because gray
  # image has one channel only.
  img_gray = cv2.cvtColor(img_bgr,cv2.COLOR_BGR2GRAY)
  #Should uncomment to process the LBP with complex network based approach
  img_gray = generate_img_by_complex_network_based_approach(img_gray, width, height)
  # Create a numpy array as
  # the same height and width
  # of RGB image
  img_lbp = np.zeros((height, width), np.uint8) # ??

  histogram = np.zeros(256)

  for i in range(0, height):
      for j in range(0, width):
          x = lbp_calculated_pixel(img_gray, i, j)
          histogram[x] = histogram[x] + 1
  result = list(histogram)
  print(result)
  return result

In [ ]:
def generate_img_by_complex_network_based_approach(img, row, col):
  for i in range(0,row):
    for j in range(0,col):
      img[i][j] = calculate_pixel_by_complex_network_based_approach(img,i,j)
  return img

In [ ]:
def calculate_pixel_by_complex_network_based_approach(img, row, col):
  start_row = max(row - 2, 0)
  start_col = max(col - 2, 0)
  end_row = min(row + 3, len(img))
  end_col = min(col + 3, len(img[0]))
  count = 0
  for i in range(start_row, end_row):
    for j in range(start_col, end_col):
      if i != row and j != col:
        result = complex_network_algorithm(img, row, col, i, j)
        if result <= 0.25 :
          count+=1

  if row - 3 >= 0:
    result = complex_network_algorithm(img, row, col, row-3, col) # Up-middle position
    if result <= 0.25:
        count+=1
  if row + 3 < len(img):
    result = complex_network_algorithm(img, row, col, row+3, col) # Down-middle position
    if result <= 0.25:
        count+=1
  if col - 3 >= 0:
    result = complex_network_algorithm(img, row, col, row, col-3) # Left-middle position
    if result <= 0.25:
        count+=1
  if col + 3 < len(img[0]):
    result = complex_network_algorithm(img, row, col, row, col+3) # Righ-Middle position
    if result <= 0.25:
        count+=1
  return count

In [ ]:
def complex_network_algorithm(matrix, row, col, row1, col1):
  return ((row-row1) ** 2 + (col-col1) ** 2 + (9/255) * (abs(matrix[row][col] - matrix[row1][col1]))) / 18

In [ ]:
def generate_histogram_from_all_files_in_directory(base_path):
  result = []
  for img in glob.glob(base_path + "/*.jpeg"):
    result.append(generate_histogram(img))
  return result

In [ ]:
def scroll_directories_to_generate_histogram_dict(base_path):
  histogram_dict = {}
  # scroll all files and directories in base path
  for dir_name in os.listdir(base_path):
      #join the current path with the name of file or directory
      complete_path = os.path.join(base_path, dir_name)
      # verify if the path is actually a directory
      if os.path.isdir(complete_path):
          print("Generating histograms from files on:", complete_path)
          histogram_dict[dir_name] = generate_histogram_from_all_files_in_directory(complete_path)
          scroll_directories_to_generate_histogram_dict(complete_path)
  return histogram_dict


In [ ]:
base_path = "YOUR_DATASET_LOCATION_HERE"
histogram_dict = scroll_directories_to_generate_histogram_dict(base_path)
np.set_printoptions(suppress=True)

print("***LBP Program is finished***")

Generating histograms from files on: /content/drive/MyDrive/I.C - Processamento de Imagens (LIBRAS)/asl_dataset/0


<ipython-input-8-50d7d3ff9a2f>:2: RuntimeWarning: overflow encountered in ubyte_scalars
  return ((row-row1) ** 2 + (col-col1) ** 2 + (9/255) * (abs(matrix[row][col] - matrix[row1][col1]))) / 18


[584.0, 66.0, 560.0, 214.0, 70.0, 10.0, 196.0, 137.0, 400.0, 47.0, 79.0, 68.0, 161.0, 30.0, 302.0, 422.0, 51.0, 17.0, 60.0, 21.0, 12.0, 2.0, 10.0, 25.0, 130.0, 30.0, 88.0, 64.0, 99.0, 9.0, 272.0, 352.0, 518.0, 92.0, 981.0, 206.0, 91.0, 21.0, 187.0, 99.0, 112.0, 13.0, 44.0, 20.0, 118.0, 21.0, 136.0, 104.0, 149.0, 29.0, 164.0, 66.0, 13.0, 5.0, 62.0, 56.0, 286.0, 43.0, 129.0, 83.0, 356.0, 49.0, 1604.0, 788.0, 51.0, 7.0, 62.0, 16.0, 12.0, 1.0, 30.0, 17.0, 54.0, 8.0, 12.0, 11.0, 24.0, 9.0, 40.0, 58.0, 16.0, 2.0, 17.0, 3.0, 1.0, 1.0, 5.0, 4.0, 18.0, 2.0, 18.0, 12.0, 14.0, 0.0, 66.0, 53.0, 153.0, 17.0, 156.0, 76.0, 42.0, 7.0, 72.0, 59.0, 50.0, 9.0, 15.0, 17.0, 63.0, 13.0, 92.0, 101.0, 160.0, 19.0, 128.0, 67.0, 15.0, 2.0, 61.0, 92.0, 421.0, 33.0, 104.0, 85.0, 344.0, 64.0, 780.0, 788.0, 412.0, 147.0, 99.0, 283.0, 38.0, 21.0, 64.0, 391.0, 450.0, 128.0, 42.0, 139.0, 136.0, 73.0, 137.0, 1921.0, 51.0, 20.0, 15.0, 44.0, 11.0, 6.0, 14.0, 46.0, 99.0, 75.0, 30.0, 104.0, 62.0, 60.0, 88.0, 719.0, 94.0, 1

KeyboardInterrupt: ignored

In [ ]:
# save the histograms generated
with open('/content/drive/MyDrive/I.C - Processamento de Imagens (LIBRAS)/histograms-v2.txt', 'w') as f:
    json.dump(histogram_dict, f)

In [ ]:
print(histogram_dict.keys())

dict_keys(['0', '1', '4', '6', '7', '9', '8', '3', '5', '2', 'b', 'g', 'f', 'h', 'a', 'i', 'c', 'e', 'j', 'd', 'n', 'r', 'q', 'p', 't', 'l', 'k', 'm', 'o', 's', 'y', 'v', 'u', 'x', 'w', 'z'])


In [ ]:
def update_matrix(matrix, row, col):
    if row - 1 >= 0:
        matrix[row - 1][col] = 1  # Posição acima
    if row + 1 < len(matrix):
        matrix[row + 1][col] = 1  # Posição abaixo
    if col - 1 >= 0:
        matrix[row][col - 1] = 1  # Posição à esquerda
    if col + 1 < len(matrix[0]):
        matrix[row][col + 1] = 1  # Posição à direita


    return matrix


# Exemplo de uso
matrix = [
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
]

row = int(input("Digite o número da linha (entre 1 e 10): "))
col = int(input("Digite o número da coluna (entre 1 e 10): "))

# Ajustando os índices para base 0
row -= 1
col -= 1

updated_matrix = update_matrix(matrix, row, col)

print("Matriz atualizada:")
for row in updated_matrix:
    print(row)

Digite o número da linha (entre 1 e 10): 5
Digite o número da coluna (entre 1 e 10): 5
Matriz atualizada:
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
